In [4]:
!pip install -Uqqq pip
!pip install -qqq langchain==0.1.5
!pip install -qqq openai==1.11.1
!pip install -Uqqq watermark==2.4.3
!pip install -Uqqq chromadb==0.4.22 
!pip install -Uqqq tiktoken==0.5.2 
!pip install -Uqqq langchain-openai==0.0.5


In [7]:
!pip install -Uqqq youtube-transcript-api==0.6.2 
!pip install -Uqqq pytube==15.0.0
!pip install -Uqqq unstructured[local-inference]==0.12.3 
!pip install -Uqqq gdown
# !pip install -Uqqq ipywidgets widgetsnbextension
!pip install -Uqqq sentence-transformers

In [19]:
import os
import textwrap

import chromadb
import langchain
import openai
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain.document_loaders import TextLoader, UnstructuredPDFLoader, YoutubeLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from dotenv import load_dotenv

In [10]:
%load_ext watermark
%watermark --iversions -v -m

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.21.0

Compiler    : Clang 14.0.0 (clang-1400.0.29.202)
OS          : Darwin
Release     : 21.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

chromadb : 0.4.22
openai   : 1.11.1
langchain: 0.1.5



In [11]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

In [13]:
# os.environ["OPENAI_API_KEY"] = "your OPENAI API KEY"
# openai.api_key = os.getenv("OPENAI_API_KEY")
load_dotenv()

True

In [16]:
!gdown 1eetuan04uj9-QKu_Vok2mbSK23G0H7yN
!gdown 1MVIhlCJS5RjVDy_s93Jb4vkHt6jAmgaa

Downloading...
From: https://drive.google.com/uc?id=1eetuan04uj9-QKu_Vok2mbSK23G0H7yN
To: /Users/jorgeboneu/code/projects/langchain_realife_applications/the-need-to-read.txt
100%|██████████████████████████████████████| 2.44k/2.44k [00:00<00:00, 5.58MB/s]
Downloading...
From: https://drive.google.com/uc?id=1MVIhlCJS5RjVDy_s93Jb4vkHt6jAmgaa
To: /Users/jorgeboneu/code/projects/langchain_realife_applications/Andrej_Karpathy_Resume.pdf
100%|███████████████████████████████████████| 46.9k/46.9k [00:00<00:00, 620kB/s]


In [17]:
txt_loader = TextLoader("./the-need-to-read.txt", encoding="utf8")

In [20]:
index = VectorstoreIndexCreator().from_loaders([txt_loader])

In [21]:
query = "Why someone in todays world would read? Answer in 3 sentences."
result = index.query_with_sources(query)
result

{'question': 'Why someone in todays world would read? Answer in 3 sentences.',
 'answer': " People in today's world would read to improve their thinking and writing skills, as well as to gain new ideas and knowledge. Reading is essential for developing critical thinking and communication skills, and it allows individuals to access a wide range of information and perspectives. ",
 'sources': './the-need-to-read.txt'}

In [22]:
print_response(result["answer"])

 People in today's world would read to improve their thinking and writing skills, as well as to gain
new ideas and knowledge. Reading is essential for developing critical thinking and communication
skills, and it allows individuals to access a wide range of information and perspectives.


## Loaders

In [23]:
yt_loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=n2uY3-2Goek", add_video_info=True
)

In [24]:
yt_documents = yt_loader.load()
yt_documents

[Document(page_content="you get more out of reading one book that's great five times than out of reading five mediocre books if your behavior doesn't change as a result of reading a book and it means you learn nothing which means it was a waste of time and many people who read books are just wasting their time because their behavior doesn't change and so I consolidate once I find something that's good I plug everything I possibly can into it and suck the juice out of it so that I can change my behavior as a result which comes from the Frameworks and how I think about it so I read one thing that's very good many times rather than trying to brag about the fact that I read a book a week because I'm like what was the book last week not that good that it wasn't worth rereading", metadata={'source': 'n2uY3-2Goek', 'title': 'How to get the most out of reading', 'description': 'Unknown', 'view_count': 41498, 'thumbnail_url': 'https://i.ytimg.com/vi/n2uY3-2Goek/hq720_2.jpg?sqp=-oaymwEoCIAKENAF8

In [25]:
document = yt_documents[0]
document.page_content

"you get more out of reading one book that's great five times than out of reading five mediocre books if your behavior doesn't change as a result of reading a book and it means you learn nothing which means it was a waste of time and many people who read books are just wasting their time because their behavior doesn't change and so I consolidate once I find something that's good I plug everything I possibly can into it and suck the juice out of it so that I can change my behavior as a result which comes from the Frameworks and how I think about it so I read one thing that's very good many times rather than trying to brag about the fact that I read a book a week because I'm like what was the book last week not that good that it wasn't worth rereading"

In [45]:
# Use OnlinePDFLoade to load PDFs from the Internets
pdf_loader = UnstructuredPDFLoader("./Andrej_Karpathy_Resume.pdf")
pdf_pages = pdf_loader.load_and_split()

In [36]:
len(pdf_pages[0].page_content)

1428

In [37]:
pdf_pages[0]

Document(page_content='Andrej Karpathy\n\nandrej.karpathy@gmail.com http://cs.stanford.edu/~karpathy/\n\nEDUCATION\n\nStanford University (PhD), 2011 –\n\nComputer Science, studying Machine Learning and Computer Vision\n\nUniversity of British Columbia (Master’s degree), 2009 - 2011\n\nComputer Science graduate studies in Machine Learning, Vision, Motor Control - Average course grade: 94.4%\n\nUniversity of Toronto (Bachelor’s degree), 2005 - 2009\n\nDouble major in Computer Science and Physics, minor in Mathematics - Cumulative GPA: 3.74\n\nWORK EXPERIENCE\n\nGoogle Research (internship), June 2011 – September 2011\n\nDeveloped learning algorithms for video classification tasks - Worked on a large-scale learning framework for video analysis\n\nTeaching Assistant\n\n- -\n\n2011: Assisted with online offering of the Machine Learning class at Stanford 2011: Graduate Probabilistic Machine Learning class 2009-2010: Taught tutorial sections for a first year Discrete Mathematics class on fou

## Text Splitters

In [46]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(pdf_pages)
len(texts)

2

In [47]:
len(texts[0].page_content), len(texts[1].page_content)

(985, 454)

In [40]:
texts[0]

Document(page_content='Andrej Karpathy\n\nandrej.karpathy@gmail.com http://cs.stanford.edu/~karpathy/\n\nEDUCATION\n\nStanford University (PhD), 2011 –\n\nComputer Science, studying Machine Learning and Computer Vision\n\nUniversity of British Columbia (Master’s degree), 2009 - 2011\n\nComputer Science graduate studies in Machine Learning, Vision, Motor Control - Average course grade: 94.4%\n\nUniversity of Toronto (Bachelor’s degree), 2005 - 2009\n\nDouble major in Computer Science and Physics, minor in Mathematics - Cumulative GPA: 3.74\n\nWORK EXPERIENCE\n\nGoogle Research (internship), June 2011 – September 2011\n\nDeveloped learning algorithms for video classification tasks - Worked on a large-scale learning framework for video analysis\n\nTeaching Assistant\n\n- -\n\n2011: Assisted with online offering of the Machine Learning class at Stanford 2011: Graduate Probabilistic Machine Learning class 2009-2010: Taught tutorial sections for a first year Discrete Mathematics class on fou

In [41]:
texts[1]

Document(page_content='COURSE WORK\n\n\n\nStanford: Machine Learning, Computer Vision, Convex Optimization, Probabilistic Graphical Models (I and II)\n\nUniversity of British Columbia: Machine Learning (I and II), Computer Vision (I and II)\n\nHACKING SKILLS\n\n\n\nPython, C++, MATLAB, Java, Objective C, Javascript/HTML/CSS, PHP, SQL\n\nINTERESTS Hobbies include Ping Pong, Ice skating, Scuba diving, PC strategy/fps games, Programming, and solving the Rubik’s cube in less than 20 seconds', metadata={'source': './Andrej_Karpathy_Resume.pdf'})

## Embeddings

In [48]:
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
hf_embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)

config_sentence_transformers.json: 100%|█████████████████████████████████████████████████████████████████████| 116/116 [00:00<00:00, 256kB/s]
README.md: 100%|████████████████████████████████████████████████████████████████████████████████████████| 10.6k/10.6k [00:00<00:00, 18.0MB/s]
sentence_bert_config.json: 100%|███████████████████████████████████████████████████████████████████████████| 53.0/53.0 [00:00<00:00, 136kB/s]
config.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 571/571 [00:00<00:00, 1.59MB/s]
pytorch_model.bin: 100%|██████████████████████████████████████████████████████████████████████████████████| 438M/438M [00:40<00:00, 10.8MB/s]
tokenizer_config.json: 100%|████████████████████████████████████████████████████████████████████████████████| 363/363 [00:00<00:00, 1.60MB/s]
vocab.txt: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 232k/232k [00:00<00:00, 1.02MB/s]
tokeni

In [49]:
text = texts[0].page_content
text

'Andrej Karpathy\n\nandrej.karpathy@gmail.com http://cs.stanford.edu/~karpathy/\n\nEDUCATION\n\nStanford University (PhD), 2011 –\n\nComputer Science, studying Machine Learning and Computer Vision\n\nUniversity of British Columbia (Master’s degree), 2009 - 2011\n\nComputer Science graduate studies in Machine Learning, Vision, Motor Control - Average course grade: 94.4%\n\nUniversity of Toronto (Bachelor’s degree), 2005 - 2009\n\nDouble major in Computer Science and Physics, minor in Mathematics - Cumulative GPA: 3.74\n\nWORK EXPERIENCE\n\nGoogle Research (internship), June 2011 – September 2011\n\nDeveloped learning algorithms for video classification tasks - Worked on a large-scale learning framework for video analysis\n\nTeaching Assistant\n\n- -\n\n2011: Assisted with online offering of the Machine Learning class at Stanford 2011: Graduate Probabilistic Machine Learning class 2009-2010: Taught tutorial sections for a first year Discrete Mathematics class on four consecutive semester

In [50]:
hf_embedding = hf_embeddings.embed_documents([text])
len(hf_embedding[0])

768

In [51]:
hf_embedding[0][:10]

[0.0018453153315931559,
 0.06233080103993416,
 -0.045333027839660645,
 -0.02713366411626339,
 0.010153532028198242,
 -0.013238377869129181,
 0.05982312560081482,
 -0.025168033316731453,
 -0.03709833323955536,
 -0.034538403153419495]

In [52]:
embeddings = OpenAIEmbeddings()

In [53]:
openai_embedding = embeddings.embed_documents([text])
len(openai_embedding[0])

1536

In [54]:
openai_embedding[0][:10]

[-0.00683751628471003,
 0.019630398629499442,
 0.018684029516211745,
 -0.0343126389652541,
 0.01570972579045955,
 0.024875985991967723,
 -0.006168298551775518,
 0.021820565924798332,
 -0.03055420067169392,
 -0.018251404061599155]

## Vectorstores

In [55]:
db = Chroma.from_documents(texts, embeddings)

In [58]:
db.similarity_search_with_score("What is the candidate work experience?", k=2)

[(Document(page_content='COURSE WORK\n\n\n\nStanford: Machine Learning, Computer Vision, Convex Optimization, Probabilistic Graphical Models (I and II)\n\nUniversity of British Columbia: Machine Learning (I and II), Computer Vision (I and II)\n\nHACKING SKILLS\n\n\n\nPython, C++, MATLAB, Java, Objective C, Javascript/HTML/CSS, PHP, SQL\n\nINTERESTS Hobbies include Ping Pong, Ice skating, Scuba diving, PC strategy/fps games, Programming, and solving the Rubik’s cube in less than 20 seconds', metadata={'source': './Andrej_Karpathy_Resume.pdf'}),
  0.47222715616226196),
 (Document(page_content='Andrej Karpathy\n\nandrej.karpathy@gmail.com http://cs.stanford.edu/~karpathy/\n\nEDUCATION\n\nStanford University (PhD), 2011 –\n\nComputer Science, studying Machine Learning and Computer Vision\n\nUniversity of British Columbia (Master’s degree), 2009 - 2011\n\nComputer Science graduate studies in Machine Learning, Vision, Motor Control - Average course grade: 94.4%\n\nUniversity of Toronto (Bach

### Storing and Loading Embeddings

In [59]:
PERSIST_DIRECTORY = "db"

db = Chroma.from_documents(
    documents=texts, embedding=embeddings, persist_directory=PERSIST_DIRECTORY
)

In [60]:
db.similarity_search_with_score("What is the candidate work experience?", k=2)

[(Document(page_content='COURSE WORK\n\n\n\nStanford: Machine Learning, Computer Vision, Convex Optimization, Probabilistic Graphical Models (I and II)\n\nUniversity of British Columbia: Machine Learning (I and II), Computer Vision (I and II)\n\nHACKING SKILLS\n\n\n\nPython, C++, MATLAB, Java, Objective C, Javascript/HTML/CSS, PHP, SQL\n\nINTERESTS Hobbies include Ping Pong, Ice skating, Scuba diving, PC strategy/fps games, Programming, and solving the Rubik’s cube in less than 20 seconds', metadata={'source': './Andrej_Karpathy_Resume.pdf'}),
  0.4730950648141497),
 (Document(page_content='Andrej Karpathy\n\nandrej.karpathy@gmail.com http://cs.stanford.edu/~karpathy/\n\nEDUCATION\n\nStanford University (PhD), 2011 –\n\nComputer Science, studying Machine Learning and Computer Vision\n\nUniversity of British Columbia (Master’s degree), 2009 - 2011\n\nComputer Science graduate studies in Machine Learning, Vision, Motor Control - Average course grade: 94.4%\n\nUniversity of Toronto (Bache

In [61]:
db.persist()

Load data

In [62]:
vectordb = Chroma(persist_directory=PERSIST_DIRECTORY, embedding_function=embeddings)

In [64]:
vectordb.similarity_search_with_score("Where does the candidate studied?", k=2)

[(Document(page_content='Andrej Karpathy\n\nandrej.karpathy@gmail.com http://cs.stanford.edu/~karpathy/\n\nEDUCATION\n\nStanford University (PhD), 2011 –\n\nComputer Science, studying Machine Learning and Computer Vision\n\nUniversity of British Columbia (Master’s degree), 2009 - 2011\n\nComputer Science graduate studies in Machine Learning, Vision, Motor Control - Average course grade: 94.4%\n\nUniversity of Toronto (Bachelor’s degree), 2005 - 2009\n\nDouble major in Computer Science and Physics, minor in Mathematics - Cumulative GPA: 3.74\n\nWORK EXPERIENCE\n\nGoogle Research (internship), June 2011 – September 2011\n\nDeveloped learning algorithms for video classification tasks - Worked on a large-scale learning framework for video analysis\n\nTeaching Assistant\n\n- -\n\n2011: Assisted with online offering of the Machine Learning class at Stanford 2011: Graduate Probabilistic Machine Learning class 2009-2010: Taught tutorial sections for a first year Discrete Mathematics class on f

## Use a Chain

In [65]:
chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
)

In [68]:
query = "Where does the candidate studied? Use no more than 2 sentences."
response = chain.run(query)

In [69]:
print_response(response)

The candidate studied at Stanford University for their PhD in Computer Science, focusing on Machine
Learning and Computer Vision. They also studied at the University of British Columbia for their
Master's degree in Computer Science, with a focus on Machine Learning and Computer Vision.


In [70]:
query = "Give a background summary of the candidate. Use no more than 3 sentences."
response = chain.run(query)
print_response(response)

Andrej Karpathy is a computer scientist with a PhD from Stanford University. He has experience in
machine learning, computer vision, and video analysis, and has worked as a research intern at
Google. He is skilled in various programming languages and has a range of hobbies including ping
pong and solving the Rubik's cube.


In [71]:
query = "Does he know how to code using nodejs?"
response = chain.run(query)
print_response(response)

Based on the provided information, there is no mention of Andrej Karpathy having experience or
knowledge in coding using Node.js.


In [72]:
query = """
How likely is this candidate to be a top-tier DL researcher 2 years from now? 
Use 0-10 scale, where
0 - chance is nonexistent
10 - beyond reasonable doubt

You must choose a number and explain why
"""
response = chain.run(query)
print_response(response)

Based on the information provided, it is difficult to accurately assess the likelihood of this
candidate becoming a top-tier DL researcher in the future. While Andrej Karpathy has a strong
educational background in computer science, with a focus on machine learning and computer vision,
and has gained experience through an internship at Google Research, it is important to consider
other factors such as research publications, contributions to the field, and collaborations with
other top-tier researchers.  Without additional information about the candidate's research
achievements, publications, and impact in the field, it is challenging to make a definitive
judgment. Therefore, I would rate the likelihood as a 5 on a scale of 0-10, indicating a neutral
stance.


## References

- [Paul Graham - The Need to Read](http://www.paulgraham.com/read.html)
- [Alex Hormozi - How to get the most out of reading](https://www.youtube.com/watch?v=n2uY3-2Goek)
- [Andrej Karpathy (very old) Resume](https://cs.stanford.edu/~karpathy/Andrej_Karpathy_Resume.pdf)
- [LangChain Docs](https://python.langchain.com/en/latest/index.html)